**Decorators**

Python has a feature that enables us to run code that wraps around a function in order to obtain some desirable side behavior.
For example

- time a function
- record every time a function is called

There are examples of packages that make heavy use of decorators.  For example, Flask is a python program that is used to create web pages. When building a web site, the user writes code for every 

**How do decorators work**

Suppose we want a function that takes as input a function (possibly with arguments) and returns a new function with possibly additional  behavior.
In the following, the additional behavior consists of printing a message before and after the function (with its arguments) was called to the console.

In [1]:
#
# function wrapper takes as input a function
# and outputs the wrapped function
# 
# note: the wrapped function returns same value as the original function
#       but when we run that function there are side effects (messages get printed)
#
def wrap_function(func):
    #
    # define function to be returned
    #
    def wrapped_function(*args, **kwargs):
        print("you called the function")
        result = func(*args, **kwargs)
        print("function returned a result:")
        print(result)
        return result
    return wrapped_function

# original function to be wrapped
def f(x):
    return(x*x)

# the wrapped function
wrapped_f=wrap_function(f)

Run the wrapped function with some argument.

In [2]:
y=wrapped_f(7)
print("y = "+str(y))

you called the function
function returned a result:
49
y = 49


The name of any function we define can be found using the __name__ attribute.

In [3]:
myfunc=print
myfunc("pay attention!")

pay attention!


In [4]:
myfunc=print
myfunc.__name__

'print'

In [5]:
myfunc=list
myfunc.__name__

'list'

In the following only slightly more complicated example we print the name of the function wrapped when it is called.

In [6]:
def wrap_function(func):
    def wrapped_function(*args, **kwargs):
        print("you called the function \""+str(func.__name__)+"\"")
        result = func(*args, **kwargs)
        print("function returned a result")
        return result
    return wrapped_function

def square(x):
    return(x*x)
def cube(x):
    return(x*x*x)

wrapped_square=wrap_function(square)
wrapped_cube=wrap_function(cube)

res=wrapped_square(7)
print(res)
res=wrapped_cube(5)
print(res)


you called the function "square"
function returned a result
49
you called the function "cube"
function returned a result
125


**Decorator as an abbreviation**

A *decorator* makes this much more concise for a user-defined functioin. 

Observe that in the examples above when we wrapped a function we created the function, then a new wrapped function 

> square -> wrapped_square

> cube -> wrapped_cube

Using the decorator modifies the behavior of the function when the function is defined so a new function is not needed, and arguably the syntax becomes simpler.

In [7]:
def wrap_function(func):
    def wrapped_function(*args, **kwargs):
        print("you called the function \""+str(func.__name__)+"\"")
        result = func(*args, **kwargs)
        print("function returned a result")
        return result
    return wrapped_function

@wrap_function
def square(x):
    return(x*x)
@wrap_function
def cube(x):
    return(x*x*x)

print(square(7))
print(cube(5))



you called the function "square"
function returned a result
49
you called the function "cube"
function returned a result
125


**Timing Functions**

To time a function we usually wrap some code around that function like in the following:

In [8]:
import time
import numpy as np

def timing_decorator(func):
    def wrapper(*args,**kwargs):
        time0=time.time()
        func(*args,**kwargs)
        time1=time.time()
        print("time to complete = "+str(round(time1-time0,5)))
    return(wrapper)

@timing_decorator
def slow_function():
    print("Slow function starting")
    time.sleep(1)
    print("Slow function executed.")

@timing_decorator
def slower_function():
    print("Slower function starting")
    time.sleep(2)
    print("Slower function executed.")

@timing_decorator
def random_duration_function():
    print("Random duration function starting")
    time.sleep(np.random.uniform(0,5))
    print("Random duration function executed.")

slow_function()
slower_function()
random_duration_function()


Slow function starting
Slow function executed.
time to complete = 1.00584
Slower function starting
Slower function executed.
time to complete = 2.00512
Random duration function starting
Random duration function executed.
time to complete = 0.49116


**Logging Calls**

In an application we might wish to log every time a certain function gets called.
So we wrap that function using a wrapper that stores a message in a file.

In [9]:
def logging_decorator(func):
    def wrapper(*args, **kwargs):
        with open("logfile.txt","a") as fout:
            fout.write(f"Calling function {func.__name__} with args: {args}, kwargs: {kwargs}")
            fout.write("\n")
        result = func(*args, **kwargs)
        with open("logfile.txt","a") as fout:
            fout.write(f"Function {func.__name__} returned {result}"+"\n")
        return result
    return wrapper

@logging_decorator
def add(x, y):
    return x + y

@logging_decorator
def times(x, y):
    return x*y


add(times(add(2, 3),add(5,6)),add(7,8))

70

**Decorators with parameters**

We might want a decorator's behavior to vary depending on the function being wrapped.

In [10]:
def wrap_function(func):
    def wrapped_function(*args, **kwargs):
        print("you called the function \""+str(func.__name__)+"\"")
        result = func(*args, **kwargs)
        print("function returned a result")
        return result
    return wrapped_function

@wrap_function
def square(x):
    return(x*x)
@wrap_function
def cube(x):
    return(x*x*x)

print(square(7))
print(cube(5))


you called the function "square"
function returned a result
49
you called the function "cube"
function returned a result
125


**isinstance**

We can use isinstance() for type checking.

In [11]:
print(isinstance(7,str))
print(isinstance("dog",str))
print(isinstance(7,int))
print(isinstance(7,float))
print(isinstance(7.,float))

False
True
True
False
True


**Functions that return decorators**

Suppose you want functions to be tested for the positional arguments having the correct type.
In the example below, we create a function that returns a decorator that takes an argument.

In [12]:
# Simple decorator with parameters
def check_type(type_of_parameter):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for ctr,arg in enumerate(args):
                if not isinstance(arg,type_of_parameter):
                    return "Incorrect type for function " + str(func.__name__)+ " in position " +str(ctr) 
            return func(*args, **kwargs)
        return wrapper
    return decorator


@check_type(str)
def join_strings(*args):
    return ''.join(args)


@check_type(int)
def add_integers(*args):
    return sum(args)


# Test the functions
print(join_strings("This","is","a","test"))
print(join_strings("This","is","my","number",71,"test"))  
print(add_integers(19, 2, 8, 533, 67, 981, 119))
print(add_integers(19, "nine",2, 8, 533,67, 981, 119))


Thisisatest
Incorrect type for function join_strings in position 4
1729
Incorrect type for function add_integers in position 1
